# Basis profile curve identification to understand electrical stimulation effects in human brain networks

Method developed in Matlab by Kai J. Miller, available at Miller KJ, Mueller KR, Hermes D. Basis profile curve identification to understand electrical stimulation effects in human brain networks. doi: https://doi.org/10.1101/2021.01.24.428020

This project was supported by the National Institute Of Mental Health of the National Institutes of Health under Award Number R01MH122258. The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health

This Jupyter Notebook was written by Tal Pal Attia, Harvey Huang, Max van den Boom and Dora Hermes (2021).

# Overview

This notebook is a walk through of the methods presented in [Miller et al. (2021)](https://www.biorxiv.org/content/10.1101/2021.01.24.428020v1).

> **Abstract (Miller et al. 2021)**
<br /> Brain networks can be explored by delivering brief pulses of electrical current in one area while measuring voltage responses in other areas. We propose a convergent paradigm to study brain dynamics, focusing on a single brain site to observe the average effect of stimulating each of many other brain sites. Viewed in this manner, visually-apparent motifs in the temporal response shape emerge from adjacent stimulation sites. This work constructs and illustrates a data-driven approach to determine characteristic spatiotemporal structure in these response shapes, summarized by a set of unique “basis profile curves” (BPCs). Each BPC may be mapped back to underlying anatomy in a natural way, quantifying projection strength from each stimulation site using simple metrics. Our technique is demonstrated for an array of implanted brain surface electrodes in a human patient. This framework enables straightforward interpretation of single-pulse brain stimulation data, and can be applied generically to explore the diverse milieu of interactions that comprise the connectome.

This notebook will walk you through the following five steps to compute and visualize Basis Profile Curves (BPCs) in an interactive manner:
1. Getting started with the Python environment and packages
2. Loading and checking the BIDS data and metadata
3. Preprocessing the data
4. Calculating BPCs 
5. Visualizing BPCs


# 1. Getting started with the Python environment and packages
Before getting started, you need to have the necessary software installed and data downloaded. 

An iEEG dataset formatted according to the Brain Imaging Data Structure used this tutorial can be downloaded from OpenNeuro:
https://openneuro.org/datasets/ds003708

## 1.1 Setting up the environment
This notebook requires the following Python packages to be installed: [bpc-requirements.txt](./bpc-requirements.txt)
To ensure all requirements are installed run 'pip install -r bpc-requirements.txt'


To easy setup an enviroment on a Mac or Linux use this file:  [bpc-dev-env.sh](./bpc-dev-env.sh) \
Run from terminal: <code>bash bpc-dev-env.sh</code> \
or from this notebook: <code>!bash bpc-dev-env.sh</code>

You can run the following cell to setup your environment for this tutorial, after running, refresh the page and make sure the bpc kernel is selected.

In [ ]:
# creating the enviroment for this toturial and refreshing the notebook
# !bash bpc-dev-env.sh

## 1.2 Importing python packages

We use several Python packages, and the following allow us to work with the BIDS dataset in this notebook:
 - [PyBIDS](https://bids-standard.github.io/pybids/#) ([Yarkoni et al., 2019](https://joss.theoj.org/papers/10.21105/joss.01294)), a Python library to centralize interactions with BIDS datasets. For more information on BIDS, see: [https://bids.neuroimaging.io](https://bids.neuroimaging.io).
 - [IeegPrep](https://pypi.org/project/ieegprep/), a python library to read, pre-process and epoch Intracranial Electroencephalography (iEEG) data that is structured according to the Brain Imaging Data Structure (BIDS)

In [ ]:
# packages importing
import os

# for scientific computing and data visualization 
import numpy as np
import pandas as pd
from ieegprep import IeegDataReader
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.decomposition import NMF
import itertools
from tqdm import tqdm

# for handling neuroimaging data
import bids
from nilearn import plotting

# for this tutorial
import functions.helperFunctions as helper
import functions.pyBPCs as pyBPCs

%matplotlib inline

# 2. Loading and checking the BIDS data and metadata

An example CCEP dataset is available on OpenNeuro ([link](https://openneuro.org/datasets/ds003708)). This dataset is formatted according to the Brain Imaging Data Structure ([BIDS](https://bids.neuroimaging.io/)) and contains one subject to work with in this tutorial. 

This dataset includes an electrocorticography (ECoG) dataset with single pulse stimulation, and accompanying metadata, such as electrode positions, channel information, stimulation events etc.

In [ ]:
# change this path to the full hardcoded BIDS data path to be analyzed
BIDS_dataset_path = '' # '/full/path/to/Basis_profile_curve/data'

## 2.1 Load BIDS metadata

We use pyBIDS to initialize a BIDSLayout: this will index the files and metadata under the specified root folder (should be 1 subject, 1 session, 1 run).

In [ ]:
# Initialize the layout
layout = bids.BIDSLayout(BIDS_dataset_path)

# Print some basic information about the layout
print(layout)
print('Subjects in this BIDS layout:', layout.get_subjects())

all_files = layout.get()
df_layout = layout.to_df()

## 2.2 Select run to analyze

User input regarding the subject, session and task to be analyzed:

In [ ]:
# Change this to make sure it fits your data
bids_sub = '01' # The subject label
bids_ses = 'ieeg01' # The session label
bids_task = 'ccep' # The task name
bids_run = '01' # The run name
bids_space = 'MNI152NLin6Sym' # The electrodes space

Next we can load filenames and metadata files needed for this analysis from the BIDSLayout:

In [ ]:
# Retrieve file names

# pyBIDS does not yet support MEF3 data, we use the OS module.
for (root, dirnames, _) in os.walk(BIDS_dataset_path):
    for directory in dirnames:
        if directory.endswith(".mefd"):
             mefName = os.path.join(root, directory)
print('We will load the following iEEG data:',mefName)
                
dataJson = layout.get(subject=bids_sub, session=bids_ses, run=bids_run, task=bids_task, suffix='ieeg', extension="json")[0]
# print('\nSidecar JSON (*_ieeg.json):', dataJson)

channels_tsv_name = layout.get(subject=bids_sub, session=bids_ses, run=bids_run, task=bids_task, suffix='channels', extension = "tsv")[0]
# print('\nChannels description (*_channels.tsv):', channels_tsv_name)

events_tsv_name = layout.get(subject=bids_sub, session=bids_ses, run=bids_run, task=bids_task, suffix='events', extension = "tsv")[0]
# print('\nevents_tsv_name:', events_tsv_name)

electrodes_tsv_name = layout.get(subject=bids_sub, session=bids_ses, suffix='electrodes', space = bids_space, extension = "tsv")[0]
# print('\nElectrode description (*_electrodes.tsv):', electrodes_tsv_name)


We use the pandas.DataFrame.head function to quickly test if our files have the expected data in them.

In [ ]:
df_channels = channels_tsv_name.get_df()  # Get file contents as a pandas DataFrame (only works for TSV files)
# np.shape(df_channels)
df_channels.head(5)

In [ ]:
df_events = events_tsv_name.get_df()  # Get file contents as a pandas DataFrame (only works for TSV files)
df_events.head()

In [ ]:
df_electrodes = electrodes_tsv_name.get_df()  # Get file contents as a pandas DataFrame (only works for TSV files)
df_electrodes.head()

## 2.3 Render brain surface

Render electrode labels in MNI space. This should open a separate browser window.

In [ ]:
marker_colors=[]

xyz_list = df_electrodes[['x', 'y','z']].values.tolist()
electrodeName_list = df_electrodes[['name']].name.values.tolist()

# gray out bad channels
for cords_name in electrodeName_list:
    if df_channels[df_channels.name == cords_name].status.values[0] == 'good':
        marker_colors.append('red')
    else:
        marker_colors.append('gray')

view = plotting.view_markers(marker_coords=xyz_list, marker_labels=electrodeName_list, marker_size=7.5,
                             marker_color=marker_colors, title='Electrodes in MNI space rendered on a standard brain') # Insert a 3d plot of markers in a brain
view.open_in_browser()

# 3. Preprocessing data

Look at the rendering with electrode labels and define the electrode of interest (e.g LMS2) and time window to be analyzed (e.g. -1,2)

In [ ]:
# define electrode of interest
el_interest = 'LMS2'
# epoch_size for visualization
epoch_limits = [-1,2]

## 3.1 Extracting relevant information from metadata
Because we have the data in BIDS, we can automatically pull information essential for analyses.

In [ ]:
# Get electrode of interest from channels.tsv 
el_interest_nr = df_channels.loc[df_channels['name'] == el_interest]

# Sampling rate from sidecar JSON (*_ieeg.json).
dict_dataJson = dataJson.get_dict()
srate = dict_dataJson["SamplingFrequency"]

# Construct a time vector
tt = pyBPCs.create_time_vector(epoch_limits,srate)

# Number of trials with electrical stimulation, good and not stimulating electrode of interest
df_include_trials = df_events.loc[(df_events["trial_type"]=='electrical_stimulation') &
                               (df_events["status"]=='good') &
                               (~df_events["electrical_stimulation_site"].str.contains(el_interest))]
df_include_trials.head()

# Filter channels to select good channels, only ECOG and SEEG
df_good_channels = df_channels[(df_channels["status"] == 'good') &
                               ((df_channels["type"] == 'ECOG') | (df_channels["type"] == 'SEEG'))] 
# df_good_channels.head()

reader = IeegDataReader(mefName, preload_data=False)

# To obtain information about time series channels we can use the Pymef function read_ts_channel_basic_info()
# ms.read_ts_channel_basic_info()

print('Sampling frequency is', srate, 'Hz')

## 3.2 Calculate convergent Matrix V

**Start running the following cell because loading all the data and re-referencing to get CCEPs takes a few minutes.**

In the meantime, we can discuss the convergent paradigm (A) used to calculate BPCs and contrast with the divergent paradigm (B) and the interpretation of the convergent paradign (E). 

![image-4.png](attachment:image-4.png)
     
    Note: Rereferencing is tailored to these CCEP data, using the custom function ccep_CAR64blocks() which applies common average referencing on CCEP data. This excludes channels with high variance from 500-2000 ms after stimulation onset or high variance from 10-100 ms after stimulation onset (channels with stimulation artifact or large evoked responses). The function also assumes that noise is shared between blocks of 64 channels.

In [ ]:
"""
We iterate through all trials with electrical stimulation, no artifacts and not stimulating the recorded electrode of interest.
"""

df_cceps = pd.DataFrame(columns = ['ccep_name_1', 'ccep_num_1', 'ccep_status_1', 'ccep_name_2', 'ccep_num_2', 'ccep_status_2'])
V_pre = []  # single-trial stimulation-evoked voltage matrix

for index, row in tqdm(df_include_trials.iloc[:].iterrows(), desc='Included trials', total=df_include_trials.shape[0]):
    
    df_data = pd.DataFrame(columns = ['channel', 'data']) # dataframe for mef3 data
    samples = [row.onset * srate + element * srate for element in epoch_limits]  # get start and end samples
    # Read mef3 data
    for channel in df_electrodes.name:
        # pyMef does not apply unit conversion to microVolt, apply Natus conversion factor 0.2658 here after pyMef
        data = 0.2658 * reader.retrieve_sample_range_data(sample_start=samples[0], sample_end=samples[1], channels=channel)[0]  # Returns 1D numpy array with data from sample X to sample Y
        curr_channel = pd.DataFrame({'channel': channel, 'data': [data]})
        df_data = pd.concat([df_data, curr_channel], ignore_index=True)  # append to mef3 data dataframe
    
    # Get stimulated channels
    trial_ccep_names = row.electrical_stimulation_site.split("-")
    curr_ccep = pd.DataFrame({ 'ccep_name_1': [trial_ccep_names[0]],
                               'ccep_num_1': [df_channels.index[df_channels['name'] == trial_ccep_names[0]].tolist()[0]],
                               'ccep_status_1': [df_channels['status'][df_channels.index[df_channels['name'] == trial_ccep_names[0]].tolist()[0]]],
                               'ccep_name_2': [trial_ccep_names[1]],
                               'ccep_num_2': [df_channels.index[df_channels['name'] == trial_ccep_names[1]].tolist()[0]],
                               'ccep_status_2': [df_channels['status'][df_channels.index[df_channels['name'] == trial_ccep_names[1]].tolist()[0]]]})
    df_cceps = pd.concat([df_cceps, curr_ccep], ignore_index=True)

    # Delete current electrical stimulation site from dataFrame
    curr_indices = df_good_channels[df_good_channels['name'].isin([trial_ccep_names[0], trial_ccep_names[1]])].index
    df_incl_channels = df_good_channels[~df_good_channels.index.isin(curr_indices)]
    
    # Run the custom function ccep_CAR64blocks() in functions/pyBPCs.py
    df_data_CAR = pyBPCs.ccep_CAR64blocks(df_data, tt, df_incl_channels)
    
    # append to single-trial stimulation-evoked voltage matrix
    V_pre.append(df_data_CAR[df_data_CAR['channel'] == el_interest]['data'].values[0])

## 3.3 Plot convergent matrix

In [ ]:
"""
Calculate baseline subtraction and plot difference
"""
V_pre = np.array(V_pre)

# calculate baseline
baseline_V = np.mean(V_pre[:,np.logical_and(tt > -.5,tt < -0.05)],1)
# make baseline into the same size
baseline_Vrep = np.transpose(np.tile(baseline_V,(np.shape(tt)[0],1)))

# subtract baseline from V_pre
V_pre_baseSub = np.subtract(V_pre,baseline_Vrep)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 10), dpi=80)

fig.suptitle('Convergent Matrix (V)',fontsize= 15)
vmin=-250
vmax=250
im = ax1.imshow(V_pre, aspect='auto',extent=helper.extents(tt) + helper.extents(df_include_trials.index),vmin=vmin, vmax=vmax)
ax1.set_xlabel('Time from stimulation (s)')
ax1.set_ylabel('Stimulation trial index (k)')
ax1.set_title('Convergent Matrix (V)')
im = ax2.imshow(V_pre_baseSub, aspect='auto',extent=helper.extents(tt) + helper.extents(df_include_trials.index),vmin=vmin, vmax=vmax)
ax2.set_xlabel('Time from stimulation (s)')
ax2.set_ylabel('Stimulation trial index (k)')
ax2.set_title('Convergent Matrix (V) after baseline correction')

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im, cax=cbar_ax)

plt.show()

# 4. Calculate BPCs

Identifying basis profile curves (BPCs) that group characteristic shapes in the convergent CCEPs. 

The input for the BPC calculation is the convergent matrix (V_pre, numpy array), the time vector (tt, numpy array), and the pandas dataframe pair_types that we will generate in this section.

## 4.1 Select time-frame for BPC extraction

In [ ]:
# select the epoch times to enter in the BPC analyses in seconds
BPCs_epoch = [0.015, 1]

## 4.2 Calculate the significance matrix

To calculate the significance matrix, we project the unit-normalized stimulation trials into all other trials. We then calculate t-values across all subgroups of stimulation pairs.

In [ ]:
# V: single-trial stimulation-evoked voltage matrix (Dimensions of V are with K total stimulation events by T total timepoints)
# tt_BPCs: BPCs time vector
V, tt_BPCs = pyBPCs.bpcVoltage(V_pre_baseSub, tt, BPCs_epoch)  # take only Voltage at BPC times

# You can compare results to taking the V_pre matrix before baseline correction, just to test
# V, tt_BPCs = pyBPCs.bpcVoltage(V_pre, tt, BPCs_epoch)  # take only Voltage at BPC times

# pair_types: structure with subgroups of stimulation pairs, with fields
#    ccep_name_1, ccep_name_2: electrodes associated with the pair, cathode-anode ordering
#    indices: indices of subgroup in V matrix (out of K total stims)

pair_types = df_cceps.groupby(['ccep_name_1', 'ccep_num_1', 'ccep_name_2', 'ccep_num_2']).size().reset_index().rename(columns={0: 'count'})
pair_types = pair_types.sort_values(by=['ccep_num_1']) 

indices_column = []
for index, row in pair_types.iterrows():
    indices_column.append(df_cceps.index[(df_cceps['ccep_name_1'] == row['ccep_name_1']) & (df_cceps['ccep_name_2'] == row['ccep_name_2'])].tolist())
pair_types['indices'] = indices_column

pair_types = pair_types.reset_index(drop=True)
# pair_types.head()

### calculate
V0 = V/(np.ones((V.shape[0], 1)) * (np.sum(V ** 2, axis=0) ** 0.5))  # normalize (L2 norm) each trial
P = V0.T @ V  # calculate internal projections

tmat = pyBPCs.nativeNormalized(pair_types, P)

# visualize the set of native normalized single stimulation cross-projections 
plt.figure(figsize=(10, 8), dpi=80)
plt.imshow(tmat,aspect='auto') # Plot a matrix or an array as an image.
plt.clim(0, 10)
plt.ylabel('stimulation pair subgroup')
plt.xlabel('stimulation pair subgroup')
plt.title('Significance matrix '+r'$\Xi$',fontsize= 15)
plt.colorbar()

plt.show()

## 4.3 Iteratively decrease inner components of non-negative matrix factorization

Using Non-Negative Matrix Factorization (NMF) on the significance matrix to cluster sites that produce similar measured responses.

In [ ]:
"""
Find two non-negative matrices (W, H) whose product approximates the factorize, 
non-negative and rescaled matrix (t0_scaled). 
At the end, the matrix H has size of number of clusters by stimulation pair sub-groups.
"""

#### Default settings and initialization

# number of cluster dimensions
cl_dim = 10
    
# number of iterations to re-run NNMF - because can get suboptimal factorization due to non-unique nature of NNMF (starts out with a random matrix)
num_reruns = 20

# convergence threshold: 1e-5
conv_thresh = 0.00001

t0 = tmat
t0[t0 < 0] = 0 # factorize t-matrix, but first make non-negative

# Globally rescale data to avoid potential overflow/underflow
t0_scaled = t0 / (np.max(t0))    

nnmf_xcorr_score = 100  # start off-diagonal penalty score out with high value

WH_struct = pd.DataFrame(columns=['W', 'H', 'nnmf_xcorr_score'])  # saving structure

#### Do the NMF

#while nnmf_xcorr_score >. 5:
while nnmf_xcorr_score > 1:
  
    cl_dim = cl_dim - 1 # reduce number of inner dimensions
    
    if cl_dim == 0:
        break

    # multiple run-throughs of  nnmf
    tmp_mat_W = []
    tmp_mat_H = []
    tmp_err = []

    for k in range(num_reruns):

        model = NMF(n_components=cl_dim, init='random', solver='mu',tol=conv_thresh,max_iter=10000)
        W = model.fit_transform(t0_scaled)
        tmp_mat_W.insert(k, W)
    
        H = model.components_
        tmp_mat_H.insert(k, H)

        rec_err = model.reconstruction_err_
        tmp_err.insert(k, rec_err)

    # select factorization with smallest error
    k_ind = np.argmin(tmp_err)    
    W_min = tmp_mat_W[k_ind]
    H_min = tmp_mat_H[k_ind]

    # Normalize so rows of H have unit norm
    Hnorms = np.sqrt(np.sum(H_min.T ** 2, axis=0))
    Hnorms_ = (np.reshape(np.tile(Hnorms.T, t0_scaled.shape[1]),(-1, Hnorms.shape[0]))).T
    H_min = H_min / Hnorms_

    # Score for this decomposition - off diagonal element weights
    HHT = H_min @ H_min.T
    HHT_triu = np.triu(HHT, 1)
    HHT_triu_reshaped = np.reshape(HHT_triu, (1,-1))
    # nnmf_xcorr_score = np.max(HHT_triu_reshaped) # max of nonnegative matrix factorization component interdependencies
    nnmf_xcorr_score = np.sum(HHT_triu_reshaped) # sum of off-diagonal nonnegative matrix factorization component interdependencies
    print('Inner dimension: ', cl_dim, ', off diagonal score: ', nnmf_xcorr_score)

    # save matrices and scores for plotting later
    curr_WH = pd.DataFrame({'W': [W_min], 'H': [H_min], 'nnmf_xcorr_score': [nnmf_xcorr_score]})
    WH_struct = pd.concat([WH_struct, curr_WH], ignore_index=True)

## 4.4 Winner take all

Each stimulation pair subgroup is assigned to one cluster.

In [ ]:
"""
winner-take-all on columns of H and then threshold by 1/(2*sqrt(length(pair_types))) 
 -- since all equal would be 1/sqrt(length(pair_types))
"""

H0 = 0 * H_min

for k in range(len(pair_types)):
    k_ind = np.argmax(H_min, axis=0)[k]
    H0[k_ind][k] = H_min[k_ind][k]

H0_ = H0 > 1 / (2 * np.sqrt(len(pair_types)))

## 4.5 Identification of Basis Profile Curves 

BPCs are identified from the clustered groups (rows of H) using linear kernel PCA.

In [ ]:
"""
output will show the subgroup numbers clustered in each BPC
"""

B = []
B_struct = pd.DataFrame(columns=['curve','pairs']) # saving structure

for q in range(H0.shape[0]):
    cl_pg = np.where(H0_[q]) # cluster pair groups
    cl_inds = [] # cluster indices (e.g. concatenated trials from cluster pair groups)
    for k in cl_pg:
        for i in (pair_types['indices'][k]).values:
            cl_inds.extend(i)

    V_tmp = V[:, cl_inds]

    E = pyBPCs.kpca(V_tmp)
    B_tmp = E[:, 0] # basis vector is 1st PC

    if np.mean(B_tmp.T @ V_tmp) < 0:
        B_tmp = -B_tmp
    
    curr_B = pd.DataFrame({'curve': [B_tmp], 'pairs': [cl_pg[0]]})
    B_struct = pd.concat([B_struct, curr_B], ignore_index=True)
    
# pairs not represented by any basis
excluded_pairs = np.where(1 - (np.sum(H0_, axis=0)))

"""
Calculate statistics for each basis curve, eliminating those where there is no significant representation
"""

B_struct = pyBPCs.curvesStatistics(B_struct, V, B, pair_types)

"""
Calculate projection weights
"""

B_struct = pyBPCs.projectionWeights(B_struct)

B_struct.pairs

# 5. Visualize BPCs

## 5.1 Plot Calculated BPCs

In [ ]:
colors = iter(cm.tab10(np.linspace(0, 1, 10)))

plt.figure(figsize=(10, 8), dpi=80)
for q in range(len(B_struct)): #  cycle through basis curves
    plt.plot(tt[tt_BPCs[0]:tt_BPCs[1]], B_struct.curve[q], color=next(colors), label=q)
    
plt.xlabel('Time from stimulation (s)')
plt.ylabel('Normalized weight of BPCs')
plt.title('Calculated BPCs',fontsize=15)
plt.legend()

plt.show()

## 5.2 Spatial representation of the BPCs

In [ ]:
marker_colors = [] # marker_color array
marker_sizes = [] # marker_size array

# all electrodes
xyz = df_electrodes[['x', 'y','z']]
xyz_list = xyz.values.tolist()
electrodeName_list = df_electrodes[['name']].name.values.tolist()
for cords in xyz_list:
    marker_colors.append('white')
    marker_sizes.append(7.5)

# el_interest
xyz_el_interest = df_electrodes.loc[df_electrodes.name == el_interest_nr.name.values[0]][['x', 'y','z']]
xyz_list.append(xyz_el_interest.values[0].tolist())
marker_colors.append('black')
marker_sizes.append(20)

pair_types['interpolated_locs'] = ''

for index, row in pair_types.iterrows():
    xyz_1 = xyz.iloc[row['ccep_num_1']].values.tolist()
    xyz_2 = xyz.iloc[row['ccep_num_2']].values.tolist()
    pair_types.at[index, 'interpolated_locs'] = np.mean((xyz_1, xyz_2), axis=0).tolist()

# non-significant stim pair sites    
interpolated_locs_list = pair_types.iloc[excluded_pairs[0]].interpolated_locs.values.tolist()
for cords in interpolated_locs_list:
    marker_colors.append('gray')
    marker_sizes.append(7.5)
    
xyz_all = [y for x in [xyz_list, interpolated_locs_list] for y in x]

# plot BPCs, colored
colors = iter(cm.tab10(np.linspace(0, 1, 10)))

for q in range(B_struct.shape[0]):
    # get electrodes for BPC
    xyz_BPC_list = pair_types.interpolated_locs[B_struct.pairs[q]].tolist()
    xyz_all = [y for x in [xyz_all, xyz_BPC_list] for y in x]
    plotweights = B_struct.plotweights[q]
    curr_color = next(colors)
    for ind_cords in range(len(xyz_BPC_list)):
        marker_colors.append(curr_color)
        marker_sizes.append(B_struct.plotweights[q][ind_cords] * 25)

# plot all
view = plotting.view_markers(marker_coords=xyz_all,marker_size=marker_sizes, # marker_labels=electrodeName_list,
                             marker_color=marker_colors, title='Spatial representation of BPCs rendered on an MNI brain') # Insert a 3d plot of markers in a brain
view.open_in_browser()

## 5.3 Optional parameters to change

Once you have completed this, you can go back to Section 4.1 and change the time interval over which the BPCs are calculated (e.g. 0.2 - 1 sec) and look at the effects on the outputs.

You can also select a different electrode in Section 3 to look at the various inputs into different regions.